# OPTUNA 300+

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, Conv2D, MaxPooling2D, BatchNormalization, Activation, Add, SeparableConv2D, MultiHeadAttention
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
from sklearn.utils.class_weight import compute_class_weight
import optuna
from optuna.integration import TFKerasPruningCallback

# Verify GPU availability
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

base_dir = 'C:\\Users\\Θάνος\\Desktop\\Thesis Thanasis\\data_aug_3'
subfolders = ['clear', 'clouds']
categories = ['Healthy_augmented', 'Damaged_augmented']
IMG_HEIGHT =  64
IMG_WIDTH = 64
BATCH_SIZE = 32

def load_data(base_dir, subfolders, categories, img_height, img_width):
    data = []
    labels = []
    image_paths = []
    for category in categories:
        class_num = categories.index(category)
        for subfolder in subfolders:
            folder_path = os.path.join(base_dir, subfolder, category)
            images = sorted(os.listdir(folder_path))
            for img_name in images:
                if img_name.endswith('.png'):
                    img_path = os.path.join(folder_path, img_name)
                    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_height, img_width))
                    img_array = tf.keras.preprocessing.image.img_to_array(img)
                    data.append(img_array)
                    labels.append(class_num)
                    image_paths.append((subfolder, category, img_name))
    return np.array(data), np.array(labels), image_paths

data, labels, image_paths = load_data(base_dir, subfolders, categories, IMG_HEIGHT, IMG_WIDTH)
data = data / 255.0

# Split data ensuring twins are in the same split
def split_data(image_paths):
    unique_image_ids = list(set([img_name for subfolder, category, img_name in image_paths]))
    train_ids, test_ids = train_test_split(unique_image_ids, test_size=0.2, random_state=42)
    train_ids, val_ids = train_test_split(train_ids, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2
    return train_ids, val_ids, test_ids

def get_split_indices(image_paths, split_ids):
    split_indices = [i for i, (subfolder, category, img_name) in enumerate(image_paths) if img_name in split_ids]
    return split_indices

train_ids, val_ids, test_ids = split_data(image_paths)
train_indices = get_split_indices(image_paths, train_ids)
val_indices = get_split_indices(image_paths, val_ids)
test_indices = get_split_indices(image_paths, test_ids)

X_train, y_train = data[train_indices], labels[train_indices]
X_val, y_val = data[val_indices], labels[val_indices]
X_test, y_test = data[test_indices], labels[test_indices]

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Validation labels shape: {y_val.shape}")
print(f"Test labels shape: {y_test.shape}")

# Define data augmentation with seed
def create_datagen(seed=None):
    return ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True,
        fill_mode='nearest'
    ), seed

datagen, seed = create_datagen(seed=42)  # Set the seed for reproducibility
datagen.fit(X_train)
train_generator = datagen.flow(X_train, y_train, batch_size=BATCH_SIZE, seed=seed)  # Use the seed here too
val_generator = datagen.flow(X_val, y_val, batch_size=BATCH_SIZE, seed=seed)

# Compute class weights using the training set
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weights = dict(enumerate(class_weights))

print(f"Class weights: {class_weights}")

# Define custom model with more complexity
def residual_block(x, filters, kernel_size):
    shortcut = x
    x = SeparableConv2D(filters, (kernel_size, kernel_size), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(filters, (kernel_size, kernel_size), padding='same')(x)
    x = BatchNormalization()(x)
    
    if shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, (1, 1), padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)
        
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def attention_block(x, filters):
    attention = MultiHeadAttention(num_heads=8, key_dim=filters)(x, x)
    attention = Add()([x, attention])
    return attention

def build_model(trial):
    num_residual_blocks = trial.suggest_int('num_residual_blocks', 2, 5)
    filters = trial.suggest_int('filters', 32, 128)
    kernel_size = trial.suggest_int('kernel_size', 3, 5)
    dense_units = trial.suggest_int('dense_units', 512, 2048)
    dropout_rate = trial.suggest_float('dropout_rate', 0.3, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    
    inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    
    x = Conv2D(filters, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2))(x)
    for _ in range(num_residual_blocks):
        x = residual_block(x, filters, kernel_size)
        x = attention_block(x, filters)
        if x.shape[1] >= 2 and x.shape[2] >= 2:
            x = MaxPooling2D((2, 2))(x)
    x = GlobalAveragePooling2D()(x)
    
    x = Dense(dense_units, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(2, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Objective function for Optuna
def objective(trial):
    model = build_model(trial)
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7, verbose=1)
    
    history = model.fit(
        train_generator,
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        epochs=50,
        validation_data=val_generator,
        class_weight=class_weights,
        callbacks=[early_stopping, reduce_lr, TFKerasPruningCallback(trial, 'val_loss')],
        verbose=1
    )
    
    val_accuracy = np.max(history.history['val_accuracy'])
    return val_accuracy

# # Run the optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=300)  # Increase trials to 300

# Print the best parameters
print("Best parameters found: ", study.best_params)
print("Best score: ", study.best_value)

# Train the best model on the full training data
best_model_params = study.best_params
best_model = build_model(optuna.trial.FixedTrial(best_model_params))

with tf.device('/GPU:0'):
    history = best_model.fit(
        train_generator,
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        epochs=100,
        validation_data=val_generator,
        class_weight=class_weights,
        callbacks=[EarlyStopping(monitor='loss', patience=10, restore_best_weights=True), ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=1e-7, verbose=1)],
        verbose=1
    )

# Save the fine-tuned model
fine_tuned_model_path = 'best_custom_model_optuna.h5'
best_model.save(fine_tuned_model_path)
print(f"Best custom model fine-tuned model saved to {fine_tuned_model_path}")

# Evaluate the model on the test set
test_predictions = best_model.predict(X_test)

# Convert one-hot encoded predictions and true labels to label indices
y_test_true = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(test_predictions, axis=1)

# Generate the confusion matrix for the test set
test_conf_matrix = confusion_matrix(y_test_true, y_test_pred)

print(f"Confusion Matrix (Test) for Best Custom Model:")
print(test_conf_matrix)

# Generate the classification report for the test set
test_class_report = classification_report(y_test_true, y_test_pred, target_names=categories)

print(f"Classification Report (Test) for Best Custom Model:")
print(test_class_report)


TensorFlow version: 2.10.0
Num GPUs Available:  1


Exception ignored in sys.unraisablehook: <built-in function unraisablehook>
Traceback (most recent call last):
  File "C:\Anaconda\envs\thesis\lib\site-packages\ipykernel\iostream.py", line 664, in write
    parent = self.parent_header
  File "C:\Anaconda\envs\thesis\lib\site-packages\ipykernel\iostream.py", line 509, in parent_header
    return self._parent_header.get()
KeyboardInterrupt: 


KeyboardInterrupt: 

##200 EPOCHS

In [2]:
# Train the best model on the full training data
best_model_params = study.best_params
best_model = build_model(optuna.trial.FixedTrial(best_model_params))

with tf.device('/GPU:0'):
    history = best_model.fit(
        train_generator,
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        epochs=200,
        validation_data=val_generator,
        class_weight=class_weights,
        callbacks=[EarlyStopping(monitor='loss', patience=10, restore_best_weights=True), ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=1e-7, verbose=1)],
        verbose=1
    )

C:\Users\NickZografos\AppData\Local\Temp\ipykernel_3228\1598057649.py:132: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)


Epoch 1/200
226/226 [==============================] - 52s 217ms/step - loss: 0.7545 - accuracy: 0.5166 - val_loss: 0.6997 - val_accuracy: 0.4197 - lr: 2.4699e-04
Epoch 2/200
226/226 [==============================] - 49s 216ms/step - loss: 0.6983 - accuracy: 0.5399 - val_loss: 0.6974 - val_accuracy: 0.4197 - lr: 2.4699e-04
Epoch 3/200
226/226 [==============================] - 49s 218ms/step - loss: 0.6872 - accuracy: 0.5478 - val_loss: 0.7009 - val_accuracy: 0.4209 - lr: 2.4699e-04
Epoch 4/200
226/226 [==============================] - 50s 221ms/step - loss: 0.6816 - accuracy: 0.5610 - val_loss: 0.6628 - val_accuracy: 0.6101 - lr: 2.4699e-04
Epoch 5/200
226/226 [==============================] - 50s 221ms/step - loss: 0.6766 - accuracy: 0.5794 - val_loss: 0.7308 - val_accuracy: 0.4855 - lr: 2.4699e-04
Epoch 6/200
226/226 [==============================] - 50s 222ms/step - loss: 0.6771 - accuracy: 0.5736 - val_loss: 1.1343 - val_accuracy: 0.5803 - lr: 2.4699e-04
Epoch 7/200
226/226 [=

In [7]:
# Save the fine-tuned model
fine_tuned_model_path = 'best_custom_model_optuna.h5'
best_model.save(fine_tuned_model_path)
print(f"Best custom model fine-tuned model saved to {fine_tuned_model_path}")

# Evaluate the model on the test set
test_predictions = best_model.predict(X_test)

# Convert one-hot encoded predictions and true labels to label indices
y_test_true = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(test_predictions, axis=1)

# Generate the confusion matrix for the test set
test_conf_matrix = confusion_matrix(y_test_true, y_test_pred)

print(f"Confusion Matrix (Test) for Best Custom Model:")
print(test_conf_matrix)

# Generate the classification report for the test set
test_class_report = classification_report(y_test_true, y_test_pred, target_names=categories)

print(f"Classification Report (Test) for Best Custom Model:")
print(test_class_report)

Best custom model fine-tuned model saved to best_custom_model_optuna.h5
76/76 [==============================] - 6s 80ms/step
Confusion Matrix (Test) for Best Custom Model:
[[1277  109]
 [ 196  834]]
Classification Report (Test) for Best Custom Model:
                   precision    recall  f1-score   support

Healthy_augmented       0.87      0.92      0.89      1386
Damaged_augmented       0.88      0.81      0.85      1030

         accuracy                           0.87      2416
        macro avg       0.88      0.87      0.87      2416
     weighted avg       0.87      0.87      0.87      2416



# TRAINING BEST MODEL PARAMETERS ON BOTH X_TRAIN & X_VAL

### Load Data, Imports and Functions

In [3]:
best_model_params = {'num_residual_blocks': 4, 'filters': 121, 'kernel_size': 5, 'dense_units': 1387, 'dropout_rate': 0.40846071442232423, 'learning_rate': 0.0002469909302048313}
best_model_params

{'num_residual_blocks': 4,
 'filters': 121,
 'kernel_size': 5,
 'dense_units': 1387,
 'dropout_rate': 0.40846071442232423,
 'learning_rate': 0.0002469909302048313}

In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, Conv2D, MaxPooling2D, BatchNormalization, Activation, Add, SeparableConv2D, MultiHeadAttention
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
from sklearn.utils.class_weight import compute_class_weight
import optuna
from optuna.integration import TFKerasPruningCallback

# Verify GPU availability
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

base_dir = 'C:\\Users\\Θάνος\\Desktop\\Thesis Thanasis\\data_aug_3'
subfolders = ['clear', 'clouds']
categories = ['Healthy_augmented', 'Damaged_augmented']
IMG_HEIGHT =  64
IMG_WIDTH = 64
BATCH_SIZE = 32

def load_data(base_dir, subfolders, categories, img_height, img_width):
    data = []
    labels = []
    image_paths = []
    for category in categories:
        class_num = categories.index(category)
        for subfolder in subfolders:
            folder_path = os.path.join(base_dir, subfolder, category)
            images = sorted(os.listdir(folder_path))
            for img_name in images:
                if img_name.endswith('.png'):
                    img_path = os.path.join(folder_path, img_name)
                    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_height, img_width))
                    img_array = tf.keras.preprocessing.image.img_to_array(img)
                    data.append(img_array)
                    labels.append(class_num)
                    image_paths.append((subfolder, category, img_name))
    return np.array(data), np.array(labels), image_paths

data, labels, image_paths = load_data(base_dir, subfolders, categories, IMG_HEIGHT, IMG_WIDTH)
data = data / 255.0

# Split data ensuring twins are in the same split
def split_data(image_paths):
    unique_image_ids = list(set([img_name for subfolder, category, img_name in image_paths]))
    train_ids, test_ids = train_test_split(unique_image_ids, test_size=0.2, random_state=42)
    train_ids, val_ids = train_test_split(train_ids, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2
    return train_ids, val_ids, test_ids

def get_split_indices(image_paths, split_ids):
    split_indices = [i for i, (subfolder, category, img_name) in enumerate(image_paths) if img_name in split_ids]
    return split_indices

train_ids, val_ids, test_ids = split_data(image_paths)
train_indices = get_split_indices(image_paths, train_ids)
val_indices = get_split_indices(image_paths, val_ids)
test_indices = get_split_indices(image_paths, test_ids)

X_train, y_train = data[train_indices], labels[train_indices]
X_val, y_val = data[val_indices], labels[val_indices]
X_test, y_test = data[test_indices], labels[test_indices]

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Validation labels shape: {y_val.shape}")
print(f"Test labels shape: {y_test.shape}")

# Define data augmentation with seed
def create_datagen(seed=None):
    return ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True,
        fill_mode='nearest'
    ), seed

datagen, seed = create_datagen(seed=42)  # Set the seed for reproducibility
datagen.fit(X_train)
train_generator = datagen.flow(X_train, y_train, batch_size=BATCH_SIZE, seed=seed)  # Use the seed here too
val_generator = datagen.flow(X_val, y_val, batch_size=BATCH_SIZE, seed=seed)

# Compute class weights using the training set
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weights = dict(enumerate(class_weights))

print(f"Class weights: {class_weights}")

# Define custom model with more complexity
def residual_block(x, filters, kernel_size):
    shortcut = x
    x = SeparableConv2D(filters, (kernel_size, kernel_size), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(filters, (kernel_size, kernel_size), padding='same')(x)
    x = BatchNormalization()(x)
    
    if shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, (1, 1), padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)
        
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def attention_block(x, filters):
    attention = MultiHeadAttention(num_heads=8, key_dim=filters)(x, x)
    attention = Add()([x, attention])
    return attention

def build_model(trial):
    num_residual_blocks = trial.suggest_int('num_residual_blocks', 2, 5)
    filters = trial.suggest_int('filters', 32, 128)
    kernel_size = trial.suggest_int('kernel_size', 3, 5)
    dense_units = trial.suggest_int('dense_units', 512, 2048)
    dropout_rate = trial.suggest_float('dropout_rate', 0.3, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    
    inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    
    x = Conv2D(filters, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2))(x)
    for _ in range(num_residual_blocks):
        x = residual_block(x, filters, kernel_size)
        x = attention_block(x, filters)
        if x.shape[1] >= 2 and x.shape[2] >= 2:
            x = MaxPooling2D((2, 2))(x)
    x = GlobalAveragePooling2D()(x)
    
    x = Dense(dense_units, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(2, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Objective function for Optuna
def objective(trial):
    model = build_model(trial)
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7, verbose=1)
    
    history = model.fit(
        train_generator,
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        epochs=50,
        validation_data=val_generator,
        class_weight=class_weights,
        callbacks=[early_stopping, reduce_lr, TFKerasPruningCallback(trial, 'val_loss')],
        verbose=1
    )
    
    val_accuracy = np.max(history.history['val_accuracy'])
    return val_accuracy

TensorFlow version: 2.10.0
Num GPUs Available:  1
Training data shape: (7248, 64, 64, 3)
Validation data shape: (2416, 64, 64, 3)
Test data shape: (2416, 64, 64, 3)
Training labels shape: (7248, 2)
Validation labels shape: (2416, 2)
Test labels shape: (2416, 2)
Class weights: {0: 0.8579545454545454, 1: 1.1984126984126984}


In [7]:
# Train the best model on the combined training and validation data
X_train_val = np.concatenate((X_train, X_val))
y_train_val = np.concatenate((y_train, y_val))

# Combine the training and validation labels
y_train_val_combined = np.concatenate((np.argmax(y_train, axis=1), np.argmax(y_val, axis=1)))

# Compute the class weights using the combined labels
class_weights_combined = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_val_combined), y=y_train_val_combined)
class_weights_combined = dict(enumerate(class_weights_combined))

print(f"Class weights for combined training and validation data: {class_weights_combined}")

datagen.fit(X_train_val)
train_val_generator = datagen.flow(X_train_val, y_train_val, batch_size=BATCH_SIZE, seed=seed)

# Define the best model with the best parameters found
best_model = build_model(optuna.trial.FixedTrial(best_model_params))

with tf.device('/GPU:0'):
    history = best_model.fit(
        train_val_generator,
        steps_per_epoch=len(X_train_val) // BATCH_SIZE,
        epochs=250,
        class_weight=class_weights_combined,
        callbacks=[EarlyStopping(monitor='loss', patience=10, restore_best_weights=True), ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=1e-7, verbose=1)],
        verbose=1
    )

# Save the fine-tuned model trained on combined data
final_model_save_path = 'best_custom_model_optuna_combined.h5'
best_model.save(final_model_save_path)
print(f"Best custom model fine-tuned model saved to {final_model_save_path}")

# Evaluate the model on the test set
test_predictions = best_model.predict(X_test)

# Convert one-hot encoded predictions and true labels to label indices
y_test_true = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(test_predictions, axis=1)

# Generate the confusion matrix for the test set
test_conf_matrix = confusion_matrix(y_test_true, y_test_pred)

print(f"Confusion Matrix (Test) for Best Custom Model:")
print(test_conf_matrix)

# Generate the classification report for the test set
test_class_report = classification_report(y_test_true, y_test_pred, target_names=categories)

print(f"Classification Report (Test) for Best Custom Model:")
print(test_class_report)

Class weights for combined training and validation data: {0: 0.861319073083779, 1: 1.1919092254563395}


C:\Users\NickZografos\AppData\Local\Temp\ipykernel_22408\624946936.py:132: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)


Epoch 1/250
302/302 [==============================] - 59s 187ms/step - loss: 0.7451 - accuracy: 0.5176 - lr: 2.4699e-04
Epoch 2/250
302/302 [==============================] - 58s 192ms/step - loss: 0.6897 - accuracy: 0.5506 - lr: 2.4699e-04
Epoch 3/250
302/302 [==============================] - 59s 194ms/step - loss: 0.6816 - accuracy: 0.5693 - lr: 2.4699e-04
Epoch 4/250
302/302 [==============================] - 59s 195ms/step - loss: 0.6790 - accuracy: 0.5840 - lr: 2.4699e-04
Epoch 5/250
302/302 [==============================] - 59s 195ms/step - loss: 0.6766 - accuracy: 0.5874 - lr: 2.4699e-04
Epoch 6/250
302/302 [==============================] - 59s 196ms/step - loss: 0.6720 - accuracy: 0.5957 - lr: 2.4699e-04
Epoch 7/250
302/302 [==============================] - 59s 196ms/step - loss: 0.6746 - accuracy: 0.5867 - lr: 2.4699e-04
Epoch 8/250
302/302 [==============================] - 59s 197ms/step - loss: 0.6748 - accuracy: 0.5855 - lr: 2.4699e-04
Epoch 9/250
302/302 [===========